In [ ]:
import chi
chi.use_site("CHI@Edge")
chi.set("project_name", "CHI-231148")

In [ ]:
from chi import container
from chi import lease


start, end = lease.lease_duration(days=2)
lease_name = "edgeracer-lease"

reservations = []
lease.add_device_reservation(reservations, count=1, device_name="iot-edgeracer-pi-01")
container_lease = lease.create_lease(lease_name, reservations)
lease_id = container_lease["id"]

print(f"created lease with name {lease_name} and uuid {lease_id}, waiting for it to start. This can take up to 60s.")
lease.wait_for_active(lease_id)
print("Done!")

In [ ]:
docker_image = "docker.io/shermanm/carrunner:3.10-jammy-20230328-latest"

container1_obj = container.create_container(
    "carrunner",
    image=docker_image,
    reservation_id=lease.get_device_reservation(container_lease["id"]),
    device_profiles=[
        "pi_camera",
        "pi_gpio",
        "pi_i2c_1"
    ],
    exposed_ports = [
        "22",
        "8887"
    ],
    command = ["/bin/bash"],
    interactive = True,
)

In [ ]:
# Detect I2c Devices. We want to see that "40" is present
cmd = "i2cdetect -r -y 1"
print(
    container.execute(container1_obj.uuid, cmd).get("output")
)

In [ ]:
# Test camera capture
cmd = "raspistill -o /tmp/o.jpg"
container.execute(container1_obj.uuid, cmd).get("output")